In [79]:
import pandas as pd
import numpy as np
import altair as alt
from pathlib import Path
import matplotlib.pyplot as plt

# App-Farben
PRIMARY_COLOR = "#1f77b4"
APP_BG = "#000000"
GRID_COLOR = "#374151"
TEXT_COLOR = "#e5e7eb"
VALUE_COLOR = "#00B8A9"

# Altair Dark Theme (wie App)
def _dark_theme():
    return {
        "config": {
            "background": APP_BG,
            "view": {"stroke": None},
            "axis": {
                "grid": True,
                "gridColor": GRID_COLOR,
                "gridOpacity": 0.20,
                "domainColor": GRID_COLOR,
                "labelColor": TEXT_COLOR,
                "titleColor": TEXT_COLOR,
                "tickColor": GRID_COLOR,
            },
            "title": {"color": TEXT_COLOR, "fontSize": 14, "anchor": "start"},
        }
    }

alt.themes.register("playerscore_dark", _dark_theme)
alt.themes.enable("playerscore_dark")

ThemeRegistry.enable('playerscore_dark')

In [80]:
ROOT = Path("..")          # hoch ins Repo-Root
PROCESSED = ROOT / "Data" / "Processed"

PROCESSED

PosixPath('../Data/Processed')

In [106]:
df_squad = pd.read_csv(PROCESSED / "squad_scores_all_seasons.csv")
df_big5  = pd.read_csv(PROCESSED / "big5_table_all_seasons.csv")

print(df_squad.shape, df_big5.shape)
df_squad.head(10)

(876, 11) (876, 22)


,Season,Squad,Min_squad,90s_squad,Age_squad_mean,NumPlayers_squad,OffScore_squad,MidScore_squad,DefScore_squad,OverallScore_squad,Comp
0,2017-2018,Alavés,34128.0,379.3,24.975445,28,311.556593,258.954138,417.066437,329.192389,es La Liga
1,2017-2018,Amiens,34114.0,379.4,26.776573,28,281.344744,275.953399,458.455720,338.584621,fr Ligue 1
2,2017-2018,Angers,34133.0,379.1,26.311751,24,356.735118,403.083943,342.899692,367.572918,fr Ligue 1
3,2017-2018,Arsenal,34125.0,379.2,25.487062,28,528.126335,540.419412,414.300162,494.281970,eng Premier League
4,2017-2018,Atalanta,34098.0,379.0,25.125403,22,478.439260,447.955786,416.023061,447.472702,it Serie A
5,2017-2018,Athletic Club,34200.0,380.2,26.545789,24,316.294307,352.944613,386.928296,352.055739,es La Liga
6,2017-2018,Atlético Madrid,34120.0,379.0,26.309965,23,452.318068,392.278240,462.832227,435.809512,es La Liga
7,2017-2018,Augsburg,30608.0,340.2,25.779763,25,342.782603,307.543390,399.199615,349.841870,de Bundesliga
8,2017-2018,Barcelona,34128.0,379.1,27.586791,23,640.057582,436.788382,389.808047,488.884671,es La Liga
9,2017-2018,Bayern Munich,30690.0,341.1,26.580515,25,591.207682,559.129751,355.752849,502.030094,de Bundesliga


In [116]:
season = "2025-2026"    # anpassen
league = "eng Premier League"   # z.B. "Bundesliga" | "Premier League" | None

df_s = df_squad[df_squad["Season"] == season].copy()
df_b = df_big5[df_big5["Season"] == season].copy()

# 👉 Liga-Filter
if league is not None:
    if "Comp" not in df_s.columns:
        raise ValueError("No 'Comp' column found in squad scores.")
    df_s = df_s[df_s["Comp"] == league].copy()

print("Squads:", df_s.shape[0])
df_s[["Squad", "Comp"]].head()

Squads: 20


,Squad,Comp
782,Arsenal,eng Premier League
783,Aston Villa,eng Premier League
793,Bournemouth,eng Premier League
794,Brentford,eng Premier League
796,Brighton,eng Premier League


In [117]:
# 1) Merge
df_rank = df_s.merge(
    df_b[["Season", "Squad", "LgRk"]],
    on=["Season", "Squad"],
    how="left"
)

# 2) saubere Spalten für Plot
df_rank["TeamScore"] = pd.to_numeric(df_rank["OverallScore_squad"], errors="coerce")
df_rank["LgRk"]      = pd.to_numeric(df_rank["LgRk"], errors="coerce")

# 3) rausfiltern
df_rank = df_rank.dropna(subset=["TeamScore", "LgRk"]).copy()

print(df_rank.shape)
df_rank[["Squad","TeamScore","LgRk"]].head()

(20, 13)


,Squad,TeamScore,LgRk
0,Arsenal,410.465602,1
1,Aston Villa,377.473999,3
2,Bournemouth,336.478016,14
3,Brentford,352.562126,15
4,Brighton,344.309751,9


In [122]:
x_domain = (250, 600)
y_domain = (1, 20)

df_plot = df_rank.copy()
df_plot["Label"] = (
    df_plot["Squad"].astype(str)
    + " ("
    + df_plot["TeamScore"].round(0).astype(int).astype(str)
    + ")"
)

base = alt.Chart(df_plot)

points = base.mark_circle(
    size=95,
    opacity=0.9,
    color=VALUE_COLOR
).encode(
    x=alt.X(
        "TeamScore:Q",
        title="Squad Score",
        scale=alt.Scale(domain=list(x_domain)),
    ),
    y=alt.Y(
        "LgRk:Q",
        title="League Rank",
        scale=alt.Scale(domain=list(y_domain), reverse=True),
    ),
    tooltip=[
        "Squad:N",
        alt.Tooltip("TeamScore:Q", title="Squad Score", format=".0f"),
        alt.Tooltip("LgRk:Q", title="League Rank", format=".0f"),
    ],
)

labels = base.mark_text(
    dx=10,
    dy=-10,
    fontSize=8,
    fontWeight="bold",
    color=TEXT_COLOR,
).encode(
    x="TeamScore:Q",
    y="LgRk:Q",
    text="Label:N",
)

footnote = alt.Chart(
    pd.DataFrame({
        "label": ["Creator: TwinAnalytics • Data: FBref / Big-5 Leagues"]
    })
).mark_text(
    align="left",
    baseline="top",
    fontSize=10.5,
    color=TEXT_COLOR,
    opacity=0.75,
).encode(
    x=alt.value(6),
    y=alt.value(700),
    text="label:N",
)

chart = (
    points
    + labels
    + footnote
).properties(
    height=600,
    width=550,
    title=f"Squad Score vs League Rank ({season})" + (f" — {league}" if league else ""),
).configure_axis(
    grid=True,
    gridOpacity=0.15,
    gridColor=GRID_COLOR,
    domain=True,
    domainColor=GRID_COLOR,
    labelColor=TEXT_COLOR,
    titleColor=TEXT_COLOR,
).configure_title(
    color=TEXT_COLOR,
    fontSize=14,
    anchor="start",
).configure_view(strokeWidth=0)

chart

alt.LayerChart(...)